# swisscoveryIA : recherche dans la collection de la bibliothèque de l'UNIGE en langage naturel

Projet réalisé dans le cadre de Hackademia 2024, Battelle 22-23 novembre 2024  

Auteurs : Antoine, Cédric, Abdoulaye, Nicolas Prongué (nicolas.prongue@unige.ch), Pablo Iriarte (pablo.iriarte@unige.ch)
Date de création : 23.11.2024  
Date de dernière modification : 22.11.2024  

* LLM : LLAMA2 -> fichier llama-2-7b-chat.Q5_K_S.gguf de 4Gb disponible sur https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF
* Briques techniques : LlamaCPP et LangChain -> bibliothèques open-source pour intéragir avec le LLM

## Fonctionnement et tâches

1. Classer la question posée dans une des deux catégories, recherche booléenne (BOOL) ou recherche conceptuelle (CONCEPT), selon la présence de certains mots dans la recherche (sac de mots)
1. Réaliser un prompt qui entoure la question posée selon le type de recherche pour arriver au meilleur résultat possible
1. Envoyer le prompt à l'IA installée en local
1. Traiter la réponse donnée par l'IA pour obtenir:
   1. BOOL : un fichier JSON avce les critères de récherche extraits par l'IA et séparés dans différents champs (auteur, date et sujet)
   2. CONCEPT : un résumé fait par l'IA et des critères de recherche avec synonymes en français et anglais pour une recherche simple
1. BOOL : Traiter la question pour ajouter un critère de recherche avec le type de document si certains mots sont présents dans la recherche ("article", "livres", etc.)
2. Envoyer la requête à l'API de swisscovery pour obtenir les 10 documents les plus pertinants (critère de pertinance propre à swisscovery)
3. Parser le fichier JSON de l'API swisscovery pour avoir une liste de résultats formatés pour les présenter sur l'interface
4. Présenter le résultat :
   1. BOOL : La liste de 10 résultats avec le lien sur chaque document vers swisscovery et donner un lien à la fin pour lancer la reqûete avancée sur swisscovery
   2. CONCEPT : Le texte de l'IA suivi d'un disclaimer de la bibliothèque qui demande à la personne de vérifier l'information avec les documents trouvés sur swisscovery et donner un lien à la fin pour lancer la reqûete simple sur swisscovery

## Résultats

Interface Web simple avec :
  * un entête
  * un petit texte d'introduction
  * un champ de recherche
  * l'espace pour les résultats
  * pied de page

In [38]:
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import multiprocessing
from datetime import datetime

# Paramètres
swisscovery_api = 'https://api-eu.hosted.exlibrisgroup.com/primo/v1/search?vid=41SLSP_UGE:VU1&tab=41SLSP_UGE_MyInst_CI&search_scope=MyInst_and_CI'
swisscovery_key = 'xxx' # Sandbox

# Recherche pour les tests 
recherche = "Je veux savoir ce qu'est le bosson de Higgs"

In [36]:
# function de chat
def chat(question):
    # Load the LlamaCpp language model, adjust GPU usage based on your hardware
    llm = LlamaCpp(
        model_path="C:/Users/pablo/AI/llms/llama-2-7b-chat.Q4_K_M.gguf",
        temperature=0.5,
        n_ctx=10000,
        n_gpu_layers=-1,
        n_batch=9000,  # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
        max_tokens=512,
        n_threads=multiprocessing.cpu_count() - 1,
        ## repeat_penalty=1.5,
        # top_p=0.5,
        verbose=False,  # Enable detailed logging for debugging
    )
    
    # Define the prompt template with a placeholder for the question
    template = """
    Question: I want to transform the question "{question}" into a new booleen query that I can use in a library catalog search.
    Add several synonyms for the subjects and the translation of the terms into french and english and combine all the termes with OR.
    Give me only the query without explanations and surrounded by [].
    
    Answer:
    """
    prompt = PromptTemplate(template=template, input_variables=["question"])
    
    # Create an LLMChain to manage interactions with the prompt and model
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    
    print("Chatbot initialized, ready to chat...")
    while True:
        # question = input("> ")
        print(datetime.now()) 
        # answer = llm_chain.run(question)
        answer = llm_chain.invoke(question)
        # print(answer, '\n')
        print("Answer done")
        print (datetime.now())
        return(answer)

In [2]:
# chat()
answer = chat(recherche)

C:\Users\pablo\AppData\Local\Temp\ipykernel_15632\2345192473.py:34: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


Chatbot initialized, ready to chat...
2024-11-22 16:58:11.143428
Answer done
2024-11-22 16:58:44.858335


In [3]:
print (answer)

{'question': "Je veux savoir ce qu'est le bosson de Higgs", 'text': ' ["Je veux savoir ce qu\'est le bosson de Higgs" OR ("matière noire" OR "particule de Higgs") OR ("boson de Higgs" OR "le boson de Higgs")]'}


In [4]:
answer['question']

"Je veux savoir ce qu'est le bosson de Higgs"

In [5]:
answer['text']

' ["Je veux savoir ce qu\'est le bosson de Higgs" OR ("matière noire" OR "particule de Higgs") OR ("boson de Higgs" OR "le boson de Higgs")]'

In [29]:
# extraction de la query
query = answer['text']
query = query.replace('`', '')
query = query.replace('´', '')
if (query[0] == '\''):
    query = query[1:]
if (query[-1] == '\''):
    query = query[:-1]
if (query[0] == ' '):
    query = query[1:]
if (query[-1] == ' '):
    query = query[:-1]
print(query)

["Je veux savoir ce qu'est le bosson de Higgs" OR ("matière noire" OR "particule de Higgs") OR ("boson de Higgs" OR "le boson de Higgs")]


In [33]:
if len(query) > 0:
    print ("Accolades OK")
    query_clean = query[1:-1]
print(query_clean)

Accolades OK
"Je veux savoir ce qu'est le bosson de Higgs" OR ("matière noire" OR "particule de Higgs") OR ("boson de Higgs" OR "le boson de Higgs")


In [34]:
# tester si il y a du bla bla avant ou après
expressions = ['note', 'recherche', 'search', 'français', 'anglais']
for expression in expressions :
    if expression in query.lower():
        print("Warning, possible bla bla dans la requête!!!")

In [40]:
# convertir en URL
import urllib.parse
query_url = swisscovery_api + '&q=' + urllib.parse.quote_plus(query) + '&apikey=' + swisscovery_key
print(query_url)

https://api-eu.hosted.exlibrisgroup.com/primo/v1/search?vid=41SLSP_UGE:VU1&tab=CentralIndex&q=any,contains,little%5B%22Je+veux+savoir+ce+qu%27est+le+bosson+de+Higgs%22+OR+%28%22mati%C3%A8re+noire%22+OR+%22particule+de+Higgs%22%29+OR+%28%22boson+de+Higgs%22+OR+%22le+boson+de+Higgs%22%29%5D&apikey=l8xxc4695843cad14404b20ea33937905cb6


In [1]:
import json
import requests
r = requests.get(query_url)
print (r.json())

NameError: name 'query_url' is not defined

In [50]:
# nombre de résultats
sw_json = r.json()
print ('Nombre de résultats dans swisscovery : ' + str(sw_json['info']['total']))

Nombre de résultats dans swisscovery : 84
